# Name:- Yash Gupta
# Roll NO. :- 2301560069
# Course :- MCA sec:-A
# Subject :- AIML

# Deduplicating data

In this notebook, we deduplicate data using the [Dedupe library](https://dedupe.readthedocs.io/en/latest/), which utilizes a shallow neural network to learn from a small training exercise.

If you are interested in building your own parser, the same folks have created the [Parserator](https://github.com/datamade/parserator) which you can use to extract text features and train your own text extraction (hooray! less brittle than regex!)

In [ ]:
import pandas as pd
import dedupe
import os

In [ ]:
customers = pd.read_csv('../data/customer_data_duped.csv', 
                        encoding='utf-8')

## Checking Data Quality

In [ ]:
customers.head()

In [ ]:
customers.dtypes

In [ ]:
for col in customers.columns:
    print(col, customers[col].isnull().sum())

## Setting up Dedupe

In [ ]:
variables = [
    {'field': 'name', 'type': 'String'},
    {'field': 'job', 'type': 'String'},
    {'field': 'company', 'type': 'String'},  
    {'field': 'street_address','type': 'String'},
    {'field': 'city','type': 'String'},
    {'field': 'state', 'type': 'String', 'has_missing': True},
    {'field': 'email', 'type': 'String', 'has_missing': True},
    {'field': 'user_name', 'type': 'String'},
]

deduper = dedupe.Dedupe(variables)

In [ ]:
deduper

In [ ]:
customers.shape

In [ ]:
deduper.sample(customers.T.to_dict(), 500)

Note: If you receive an error like this:

```/usr/local/lib/python2.7/site-packages/dedupe/sampling.py:39: UserWarning: 250 blocked samples were requested, but only able to sample 249
  % (sample_size, len(blocked_sample)))
```

you can continue (some were selected), or use the suggested number (^ here it would be 249)

#### Either use training file (uncomment) or resume active training below

In [ ]:
training_file = '../data/ignore-dedupe-training.json'
#if os.path.exists(training_file):
#    with open(training_file, 'rb') as f:
#        deduper.readTraining(f)

In [ ]:
dedupe.consoleLabel(deduper)

In [ ]:
deduper.train()

In [ ]:
with open(training_file, 'w') as tf:
    deduper.writeTraining(tf)

In [ ]:
dupes = deduper.match(customers.T.to_dict())

In [ ]:
dupes

In [ ]:
dupes[2]

In [ ]:
customers.iloc[[741,1107]]

### Exercise: Flag duplicates by adding 2 extra columns, one for confidence score and one for duplicate_ids

In [ ]:
# %load ../solutions/dedupe.py


In [ ]:
customers[customers.confidence.notnull() == True].head()

# String Matching

In this notebook, we use [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy), a popular string matching library by SeatGeek. 

For more information on the different methods available and how they differ, see [their blog post explaining methodologies](http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/).

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
berlin = ['Berlin, Germany', 
          'Berlin, Deutschland', 
          'Berlin', 
          'Berlin, DE']

#### Try matching the first and second strings: 'Berlin, Germany' and 'Berlin, Deutschland'

In [ ]:
fuzz.partial_ratio(berlin[0], berlin[1])

In [ ]:
fuzz.ratio?

In [ ]:
fuzz.ratio(berlin[0], berlin[1])

In [ ]:
fuzz.token_set_ratio(berlin[0], berlin[1])

In [ ]:
fuzz.token_sort_ratio(berlin[0], berlin[1])

#### Try matching the second and third strings: 'Berlin, Deutschland' and 'Berlin'

In [ ]:
fuzz.partial_ratio(berlin[1], berlin[2])

In [ ]:
fuzz.ratio(berlin[1], berlin[2])

In [ ]:
fuzz.token_sort_ratio(berlin[1], berlin[2])

### What do you think will score lowest and highest for the final two elements: 
- 'Berlin'
- 'Berlin, DE'

In [ ]:
fuzz.token_set_ratio(berlin[2], berlin[3])

### Extracting a guess out of a list

In [ ]:
choices = ['Germany', 'Deutschland', 'France', 
           'United Kingdom', 'Great Britain', 
           'United States']

In [ ]:
process.extract('DE', choices, limit=2)

In [ ]:
process.extract('UK', choices)

In [ ]:
process.extract('frankreich', choices)

### Will this properly extract?

In [ ]:
process.extract('U.S.', choices)

# Managing Nulls with Pandas

In this notebook, we will take a look at some ways to manage nulls using Pandas DataFrames.

For even more details on how to do this, check out the [Panda's documentation](http://pandas.pydata.org/pandas-docs/stable/missing_data.html).

In [ ]:
import pandas as pd
from numpy import random

In [ ]:
df = pd.read_csv('../data/iot_example_with_nulls.csv')

### Data Quality Check

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.note.value_counts()

### Let's remove all null values (including the note: n/a)

In [ ]:
df = pd.read_csv('../data/iot_example_with_nulls.csv', 
                 na_values=['n/a'])

### Test to see if we can use dropna

In [ ]:
df.shape

In [ ]:
df.dropna().shape

In [ ]:
df.dropna(how='all', axis=1).shape

### Test to see if we can drop columns

In [ ]:
my_columns = list(df.columns)

In [ ]:
my_columns

In [ ]:
list(df.dropna(thresh=int(df.shape[0] * .9), axis=1).columns)

### I want to find all columns that have missing data

In [ ]:
missing_info = list(df.columns[df.isnull().any()])

In [ ]:
missing_info

In [ ]:
for col in missing_info:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))

In [ ]:
for col in missing_info:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print('percent missing for column {}: {}'.format(
        col, percent_missing))

### Can I easily substitute majority values in for missing data?

In [ ]:
df.note.value_counts()

In [ ]:
df.build.value_counts().head()

In [ ]:
df.latest.value_counts()

In [ ]:
df.latest = df.latest.fillna(0)

### Have not yet addressed temperature missing values... Let's find a way to fill

In [ ]:
df.username.value_counts().head()

In [ ]:
df = df.set_index('timestamp')

In [ ]:
df.head()

In [ ]:
df.temperature = df.groupby('username').temperature.fillna(
    method='backfill', limit=3)

### Exercise: How many temperature values did I fill? What percentage of values are still missing (for temperature)?

In [ ]:
# %load ../solutions/nulls.py


In [ ]:
rows_filled

In [ ]:
still_missing

# Preprocessing with Scikit-learn

# Scikit Learn Preprocessing

In this notebook, we'll use `sklearn.preprocessing` to do some scaling for us. If you need to prepare data for machine learning or feature extraction, the [sklearn.preprocessing documentation](http://scikit-learn.org/stable/modules/preprocessing.html) has great examples.

In [ ]:
from sklearn import preprocessing
import pandas as pd
from datetime import datetime

In [ ]:
hvac = pd.read_csv('../data/HVAC_with_nulls.csv')

## Checking Data Quality

In [ ]:
hvac.dtypes

In [ ]:
hvac.shape

In [ ]:
hvac.head()

## Impute missing values with mean

In [ ]:
imp = preprocessing.Imputer(missing_values='NaN', 
                            strategy='mean')

In [ ]:
hvac_numeric = hvac[['TargetTemp', 'SystemAge']]

In [ ]:
imp = imp.fit(hvac_numeric.loc[:10])

In [ ]:
transformed = imp.fit_transform(hvac_numeric)

In [ ]:
transformed

In [ ]:
hvac['TargetTemp'], hvac['SystemAge'] = transformed[:,0], transformed[:,1]

In [ ]:
hvac.head()

## Scale temperature values

In [ ]:
hvac['ScaledTemp'] = preprocessing.scale(hvac['ActualTemp'])

In [ ]:
hvac['ScaledTemp'].head()

## Scale using a min and max scaler

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
temp_minmax = min_max_scaler.fit_transform(hvac[['ActualTemp']])

In [ ]:
temp_minmax

### Exercise: add the `temp_minmax` back to the dataframe as a new column

In [ ]:
# %load ../solutions/preprocessing.py



# Case Study: Preparing Lobste.rs Stories for Machine Learning

In this case study, we'll be preparing [lobste.rs](http://lobste.rs) stories for machine learning. To do so, we need to extract features and clean up the messy parts of the data. We'll be using Pandas along with `sklearn.preprocessing` and `fuzzywuzzy`. 

In [ ]:
import pandas as pd
import requests
from fuzzywuzzy import fuzz
from collections import Counter
from sklearn import preprocessing

### If you'd rather read from the API to get the latest, uncomment the details (and add comment to the final line)

In [ ]:
#resp = requests.get('https://lobste.rs/hottest.json')
#stories = pd.read_json(resp.content)
#stories = stories.set_index('short_id')

stories = pd.read_json('../data/all_lobsters.json')

In [ ]:
stories.head()

In [ ]:
stories.dtypes

### Let's take a look at the submitter_user field, as it appears like a dict

In [ ]:
stories.submitter_user.iloc[3]

In [ ]:
user_df = stories['submitter_user'].apply(pd.Series)

In [ ]:
user_df.head()

### Can we combine the user data without potential column overlap?

In [ ]:
set(user_df.columns).intersection(stories.columns)

In [ ]:
user_df = user_df.rename(columns={'created_at': 
                                  'user_created_at'})

In [ ]:
stories = pd.concat([stories.drop(['submitter_user'], axis=1), 
                     user_df], axis=1)

In [ ]:
stories.head()

### Let's check for nulls

In [ ]:
stories.shape

In [ ]:
stories.dropna().shape

In [ ]:
stories.dropna(thresh=10, axis=1).shape

### Exercise: which columns would be dropped?

In [ ]:
# %load ../solutions/lobsters_dropped.py


## Let's make the tags easier to use by having them as features in the columns.

In [ ]:
tag_df = stories.tags.apply(pd.Series)

In [ ]:
tag_df.head()

In [ ]:
# what are our unique tags?

pd.unique(tag_df.values.ravel())

In [ ]:
set(tag_df.values.ravel())

In [ ]:
len(pd.unique(tag_df.values.ravel()))

In [ ]:
# most common tags

Counter(tag_df.values.ravel()).most_common(5)

### Let's create a dummy df with our tags

In [ ]:
tag_df = pd.get_dummies(
    tag_df.apply(pd.Series).stack()).sum(level=0)

In [ ]:
tag_df.head()

### Now we can add it back to our stories DataFrame

In [ ]:
stories = pd.concat([stories.drop('tags', axis=1), 
                     tag_df], axis=1)

In [ ]:
stories.head()

### Another potentially useful feature is the post times...

In [ ]:
stories['created_hour'] = stories.created_at.map(
    lambda x: x.hour)

In [ ]:
stories['created_dow'] = stories.created_at.map(
    lambda x: x.weekday())

### Let's analyze some of the correlations in our features so far...

In [ ]:
stories[['created_hour', 'score']].corr()

In [ ]:
stories[['created_dow', 'score']].corr()

In [ ]:
stories[['karma', 'score']].corr()

In [ ]:
stories[['comment_count', 'score']].corr()

In [ ]:
stories[['hardware', 'score']].corr()

## Exercise: can you find a more highly positive correlation?

## We might also want/need to normalize scores. We can use a Scaler / MinMaxScaler or Normalizer

In [ ]:
normed_score = preprocessing.normalize(stories[['score']])

In [ ]:
normed_score[:5]

#### hmm... maybe a min-max scaler works better for our needs!

In [ ]:
scaler = preprocessing.MinMaxScaler()

In [ ]:
scaled_score = scaler.fit_transform(stories[['score']])

In [ ]:
scaled_score[:5]

In [ ]:
stories['scaled_score'] = scaled_score[:,0]

# Dask Pipeline

## What else should we add?

- fuzzywuzzy to find match of title with topics
- add normalization or scaling to comments
- extract domain name
- number of words in the title
- number of capitalized words in the title
- use NLP to extract named entities from the title
- what else?

## Tracking the International Space Station with Dask

In this notebook, we will use two APIs: [Google Maps Geocoder](https://developers.google.com/maps/documentation/geocoding/) and the [open notify API for ISS location](http://api.open-notify.org/). We will use them to track the ISS location and next pass time in relation to a list of cities.

To help build our graphs and intelligently parallelize data, we will use [Dask](http://dask.pydata.org/en/latest/), specifically [Dask delayed](http://dask.pydata.org/en/latest/delayed.html).

In [ ]:
import requests
import logging
import sys
import numpy as np
from time import sleep
from datetime import datetime
from math import radians
from dask import delayed
from operator import itemgetter
from sklearn.neighbors import DistanceMetric

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### First, we need to get lat and long pairs from a list of cities

In [ ]:
def get_lat_long(address):
    resp = requests.get(
        'https://eu1.locationiq.org/v1/search.php',
        params={'key': '92e7ba84cf3465', #Please be kind, you can generate your own for more use here - https://locationiq.org :D
                'q': address,
                'format': 'json'}
    )
    if resp.status_code != 200:
        print('There was a problem with your request!')
        print(resp.content)
        return
    data = resp.json()[0]
    return {
        'name': data.get('display_name'),
        'lat': float(data.get('lat')),
        'long': float(data.get('lon')),
    }

In [ ]:
get_lat_long('Berlin, Germany')

In [ ]:
locations = []
for city in ['Seattle, Washington', 'Miami, Florida', 
             'Berlin, Germany', 'Singapore', 
             'Wellington, New Zealand',
             'Beirut, Lebanon', 'Beijing, China', 'Nairobi, Kenya',
             'Cape Town, South Africa', 'Buenos Aires, Argentina']:
    locations.append(get_lat_long(city))
    sleep(2)

In [ ]:
locations

### Now we can define the functions we will use to get the ISS data and compare location and next pass times amongst cities 

In [ ]:
def get_spaceship_location():
    resp = requests.get('http://api.open-notify.org/iss-now.json')
    location = resp.json()['iss_position']
    return {'lat': float(location.get('latitude')),
            'long': float(location.get('longitude'))}

In [ ]:
def great_circle_dist(lon1, lat1, lon2, lat2):
    "Found on SO: http://stackoverflow.com/a/41858332/380442"
    dist = DistanceMetric.get_metric('haversine')
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    X = [[lat1, lon1], [lat2, lon2]]
    kms = 6367
    return (kms * dist.pairwise(X)).max()

In [ ]:
def iss_dist_from_loc(issloc, loc):
    distance = great_circle_dist(issloc.get('long'), 
                                 issloc.get('lat'), 
                                 loc.get('long'), loc.get('lat'))
    logging.info('ISS is ~%dkm from %s', int(distance), loc.get('name'))
    return distance

In [ ]:
def iss_pass_near_loc(loc):
    resp = requests.get('http://api.open-notify.org/iss-pass.json',
                        params={'lat': loc.get('lat'), 
                                'lon': loc.get('long')})
    data = resp.json().get('response')[0]
    td = datetime.fromtimestamp(data.get('risetime')) - datetime.now()
    m, s = divmod(int(td.total_seconds()), 60)
    h, m = divmod(m, 60)
    logging.info('ISS will pass near %s in %02d:%02d:%02d',loc.get('name'), h, m, s)
    return td.total_seconds()

In [ ]:
iss_dist_from_loc(get_spaceship_location(), locations[4])

In [ ]:
iss_pass_near_loc(locations[4])

### Let's create a delayed pipeline

In [ ]:
output = []

for loc in locations:
    issloc = delayed(get_spaceship_location)()
    dist = delayed(iss_dist_from_loc)(issloc, loc)
    output.append((loc.get('name'), dist))

closest = delayed(lambda x: sorted(x, 
                                   key=itemgetter(1))[0])(output)

In [ ]:
closest

### Let's see our DAG!

In [ ]:
closest.visualize()

### Remember: it is lazy, so let's start it with `compute()`

In [ ]:
closest.compute()

### Exercise: which city will it fly over next?

### Extra: add your city and compare!

In [ ]:
# %load ../solutions/dask.py

